In [1]:
!pip install numpy pandas scikit-learn kmodes seaborn matplotlib

  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached kmodes-0.12.2-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.8-cp311-cp311-win_amd64.whl.metadata (52 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.0-cp311-cp311-win_amd64.whl.metadata (115 kB)
  Using cached kiwisol

In [22]:
import pandas as pd
import numpy as np
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from glob import glob

BASE_DIR = Path().resolve().parent.parent.parent
base_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" 
data_path = base_path / "data"
save_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "3.여행루트추천" / "model"
gps_data_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "3.여행루트추천" / "data"
# ---------------------------------------------------------
# 1. 데이터 로드 & 기본 전처리
# ---------------------------------------------------------

df = pd.read_csv(data_path / "rating_user_info_all.csv")
gps_data_list = glob(str(gps_data_path) + "/*.csv")

In [27]:
traveler_id_list = [csv_path.split('\\')[-1].split('.')[0] for csv_path in gps_data_list]

In [2]:
df["SEASON"].value_counts()


SEASON
summer    2234
autumn     332
spring     314
Name: count, dtype: int64

In [23]:
print("gps data len : ", len(gps_data_list))

gps data len :  1675


In [24]:
# 제주인 user만 filtering
df = df[df["TRAVEL_STATUS_DESTINATION"]=="제주"]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1681 entries, 1 to 2878
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   GENDER                     1681 non-null   object
 1   AGE_GRP                    1681 non-null   int64 
 2   MARR_STTS                  1681 non-null   int64 
 3   JOB_NM                     1681 non-null   int64 
 4   INCOME                     1681 non-null   int64 
 5   TRAVEL_NUM                 1681 non-null   int64 
 6   TRAVEL_STYL_1              1681 non-null   int64 
 7   TRAVEL_STATUS_RESIDENCE    1681 non-null   object
 8   TRAVEL_STATUS_DESTINATION  1681 non-null   object
 9   TRAVEL_STATUS_ACCOMPANY    1681 non-null   object
 10  TRAVEL_MOTIVE_1            1681 non-null   int64 
 11  TRAVEL_COMPANIONS_NUM      1681 non-null   int64 
 12  MONTH                      1681 non-null   int64 
 13  SEASON                     1681 non-null   object
 14  HOW_LONG     

In [29]:
# gps data가 없는 행 삭제
df = df[df['TRAVELER_ID'].isin(traveler_id_list)].reset_index(drop=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 1675 non-null   int64 
 1   TRAVELER_ID                1675 non-null   object
 2   GENDER                     1675 non-null   object
 3   AGE_GRP                    1675 non-null   int64 
 4   MARR_STTS                  1675 non-null   int64 
 5   JOB_NM                     1675 non-null   int64 
 6   INCOME                     1675 non-null   int64 
 7   TRAVEL_NUM                 1675 non-null   int64 
 8   TRAVEL_STYL_1              1675 non-null   int64 
 9   TRAVEL_STATUS_RESIDENCE    1675 non-null   object
 10  TRAVEL_STATUS_DESTINATION  1675 non-null   object
 11  TRAVEL_STATUS_ACCOMPANY    1675 non-null   object
 12  TRAVEL_MOTIVE_1            1675 non-null   int64 
 13  TRAVEL_COMPANIONS_NUM      1675 non-null   int64 
 14  MONTH   

In [31]:
# TRAVELER_ID 등 불필요 컬럼 제거
traveler_id = df["TRAVELER_ID"]
drop_cols = [col for col in ["TRAVELER_ID", "Unnamed: 0"] if col in df.columns]
df = df.drop(columns=drop_cols)

# -----------------------------
# 범주형 / 수치형 구분
# -----------------------------
cat_cols = [
    "GENDER",
    "AGE_GRP",
    "MARR_STTS",
    "JOB_NM",
    "TRAVEL_STYL_1",
    "TRAVEL_STATUS_RESIDENCE",
    "TRAVEL_STATUS_ACCOMPANY",
    "TRAVEL_MOTIVE_1",
    "SEASON",
    "MONTH",
]
num_cols = [
    "TRAVEL_NUM",
    "TRAVEL_COMPANIONS_NUM",
    "HOW_LONG",
    "INCOME",
    
]

# 결측값 처리
df[cat_cols] = df[cat_cols].fillna("Unknown")
for c in num_cols:
    df[c] = df[c].fillna(df[c].mode())

In [32]:
# ---------------------------------------------------------
# 2. 수치형 변수 스케일링
# ---------------------------------------------------------

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df[num_cols])

for col in cat_cols:
    df_scaled[col] = df_scaled[col].astype(str)


In [34]:
df_scaled.head()

,GENDER,AGE_GRP,MARR_STTS,JOB_NM,INCOME,TRAVEL_NUM,TRAVEL_STYL_1,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,MONTH,SEASON,HOW_LONG
0,남,40,2,1,2.288057,-0.381098,1,제주특별자치도,제주,자녀 동반 여행,8,0.434275,9,autumn,-1.687884
1,여,50,1,2,0.144303,-0.822244,3,제주특별자치도,제주,나홀로 여행,6,-1.041202,8,summer,-1.687884
2,여,20,1,2,1.216180,-0.381098,4,대구광역시,제주,나홀로 여행,1,-1.041202,9,autumn,-0.912458
3,여,30,1,12,-1.463511,-0.822244,3,부산광역시,제주,2인 여행(가족 외),3,-0.303464,9,autumn,-0.137031
4,여,20,1,3,-0.391635,1.824633,1,광주광역시,제주,나홀로 여행,4,-1.041202,8,summer,1.413823


In [53]:

# ---------------------------------------------------------
# 3. K-Prototypes 군집화
# ---------------------------------------------------------

# 입력은 numpy 배열 형태여야 함
data_np = df_scaled[cat_cols + num_cols].to_numpy(dtype=object)


# 범주형 인덱스 (KPrototypes 요구사항)
cat_idx = list(range(len(cat_cols)))   # 0 ~ len(cat_cols)-1



# 군집 개수 설정 (원하면 자동 튜닝 가능)
K = 8

kproto = KPrototypes(n_clusters=K, init='Huang', random_state=1004, n_init=3)
clusters = kproto.fit_predict(data_np, categorical=cat_idx)

df_scaled["cluster"] = clusters

print("군집 분포:\n", df_scaled["cluster"].value_counts())


군집 분포:
 cluster
4    464
6    363
7    227
3    178
2    148
1    139
5    118
0     38
Name: count, dtype: int64


In [54]:

# ---------------------------------------------------------
# 4. 군집 요약 통계 생성 (자동 라벨링용)
# ---------------------------------------------------------

def summarize_cluster(df_cluster):
    summary = {}

    # 범주형
    for col in cat_cols:
        vc = df_cluster[col].value_counts(normalize=True)
        if len(vc) == 0:
            summary[col] = {"top": "Unknown", "ratio": 0.0}
        else:
            summary[col] = {"top": vc.index[0], "ratio": float(vc.iloc[0])}

    # 수치형 (scaled mean 저장)
    for col in num_cols:
        vals = df_cluster[col].dropna()
        summary[col] = {"mean": float(vals.mean())} if len(vals) else {"mean": 0.0}

    return summary


def safe_get(summary, col, key, default):
    try:
        return summary[col][key]
    except:
        return default

cluster_summaries = {
    c: summarize_cluster(df_scaled[df_scaled["cluster"] == c])
    for c in sorted(df_scaled["cluster"].unique())
}


In [55]:

# ---------------------------------------------------------
# 5. 군집 라벨 자동 생성
# ---------------------------------------------------------

style_map = {
    "1": "자연 매우선호", "2": "자연 중간선호", "3": "자연 약간선호",
    "4": "중립", "5": "도시 약간선호", "6": "도시 중간선호", "7": "도시 매우선호"
}

season_map = {
    "spring": "봄", "summer": "여름", "autumn": "가을", "winter": "겨울"
}


def make_label(summary):
    # ---------------------------------------------------
    # 1) 수치형 mean을 역스케일
    # ---------------------------------------------------
    # safe_get으로 scaled_mean 불러오기 (결측 대비)
    scaled_means = [
        safe_get(summary, col, "mean", 0) 
        for col in num_cols
    ]

    # inverse scaling
    raw_means = scaler.inverse_transform([scaled_means])[0]

    # dict로 매핑
    raw_mean_dict = {
        col: raw_means[i] 
        for i, col in enumerate(num_cols)
    }

    # ---------------------------------------------------
    # 2) 범주형 요약값 (top) 안전하게 불러오기
    # ---------------------------------------------------
    age = safe_get(summary, "AGE_GRP", "top", "Unknown")
    accompany = safe_get(summary, "TRAVEL_STATUS_ACCOMPANY", "top", "Unknown")

    style_code = str(safe_get(summary, "TRAVEL_STYL_1", "top", "0"))
    style = style_map.get(style_code, f"스타일{style_code}")

    season_raw = safe_get(summary, "SEASON", "top", "Unknown")
    season = season_map.get(season_raw, season_raw)

    # ---------------------------------------------------
    # 3) 역스케일된 HOW_LONG 사용
    # ---------------------------------------------------
    how_long = round(raw_mean_dict["HOW_LONG"], 1)

    # ---------------------------------------------------
    # 4) 최종 라벨 문장 생성
    # ---------------------------------------------------
    return f"{age}대 · {accompany} · {style} · 평균 {how_long}일 여행자"




cluster_labels = {
    c: make_label(summary)
    for c, summary in cluster_summaries.items()
}

print("\n=== 생성된 군집 라벨 ===")
for c, label in cluster_labels.items():
    print(f"[Cluster {c}] {label}")


=== 생성된 군집 라벨 ===
[Cluster 0] 40대 · 3대 동반 여행(친척 포함) · 자연 매우선호 · 평균 3.8일 여행자
[Cluster 1] 30대 · 자녀 동반 여행 · 자연 매우선호 · 평균 1.3일 여행자
[Cluster 2] 20대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.3일 여행자
[Cluster 3] 30대 · 나홀로 여행 · 중립 · 평균 1.3일 여행자
[Cluster 4] 30대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.6일 여행자
[Cluster 5] 50대 · 자녀 동반 여행 · 중립 · 평균 3.9일 여행자
[Cluster 6] 20대 · 2인 여행(가족 외) · 중립 · 평균 3.5일 여행자
[Cluster 7] 30대 · 자녀 동반 여행 · 중립 · 평균 3.9일 여행자


In [56]:


# ---------------------------------------------------------
# 6. 신규 유저 cluster assign 함수
# ---------------------------------------------------------

def assign_new_user(user_dict):
    """
    user_dict 예시:
    {
        "GENDER": "남",
        "AGE_GRP": 30,
        "MARR_STTS": 1,
        "JOB_NM": 3,
        ...
    }
    """
    new_df = pd.DataFrame([user_dict])

    # 결측 보정
    new_df[cat_cols] = new_df[cat_cols].fillna("Unknown")
    for c in num_cols:
        new_df[c] = new_df[c].fillna(df[c].median())

    # 스케일링
    new_df[num_cols] = scaler.transform(new_df[num_cols])

    # numpy 변환
    new_np = new_df[cat_cols + num_cols].to_numpy()

    # 군집 예측
    cluster = kproto.predict(new_np, categorical=cat_idx)[0]
    label = cluster_labels[cluster]

    return cluster, label



In [57]:

# ---------------------------------------------------------
# 7. 신규 유저 테스트
# ---------------------------------------------------------

example_user = {
    "GENDER": "남",
    "AGE_GRP": 30,
    "MARR_STTS": 1,
    "JOB_NM": 3,
    "INCOME": 4,
    "TRAVEL_STYL_1": 2,
    "TRAVEL_STATUS_RESIDENCE": "서울특별시",
    "TRAVEL_STATUS_ACCOMPANY": "2인 여행(가족 외)",
    "TRAVEL_MOTIVE_1": 7,
    "TRAVEL_NUM": 3,
    "TRAVEL_COMPANIONS_NUM": 1,
    "MONTH": 8,
    "SEASON": "summer",
    "HOW_LONG": 3,
}

cluster_id, label = assign_new_user(example_user)
print("\n신규 유저 군집:", cluster_id)
print("군집 라벨:", label)



신규 유저 군집: 4
군집 라벨: 30대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.6일 여행자


=== 생성된 군집 라벨 ===
- [Cluster 0] 40대 · 3대 동반 여행(친척 포함) · 자연 매우선호 · 평균 3.8일 여행자
- [Cluster 1] 30대 · 자녀 동반 여행 · 자연 매우선호 · 평균 1.3일 여행자
- [Cluster 2] 20대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.3일 여행자
- [Cluster 3] 30대 · 나홀로 여행 · 중립 · 평균 1.3일 여행자
- [Cluster 4] 30대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.6일 여행자
- [Cluster 5] 50대 · 자녀 동반 여행 · 중립 · 평균 3.9일 여행자
- [Cluster 6] 20대 · 2인 여행(가족 외) · 중립 · 평균 3.5일 여행자
- [Cluster 7] 30대 · 자녀 동반 여행 · 중립 · 평균 3.9일 여행자

In [63]:
# cluster_name_map = {
#     0: "여유롭게 여행해 듀오",
#     1: "짧게 여행해 듀오",
#     2: "초단기 민첩 여행러",
#     3: "패밀리 힐링러",
#     4: "뉴트럴 듀오 탐색형 여행자",
#     5: "자연애호 패밀리 스프린터",
#     6: "자연애호 그룹 여행러",
#     7: "표준형 듀오 여행자",
#     8: "자연에서 충전해 듀오",
#     9: "패밀리 단기 힐링러",
# }
cluster_name_map = {
    0: "자연에 진심인 힐링패밀리",
    1: "자연집중 번개여행 패밀리",
    2: "탐험 ON한 듀오 트래블러",
    3: "솔로감성 잠깐 리프레시러",
    4: "느슨하게 떠나는 듀오 무드러",
    5: "차분템포 자연수집 패밀리",
    6: "뉴트럴바이브 듀오 스테이러",
    7: "패밀리 무드 ON 스탠다드러",
}

In [64]:
from joblib import dump

# 수치형 median 미리 저장 (추론 시 df 없이 사용)
num_medians = {c: float(df[c].median()) for c in num_cols}

artifacts = {
    "kproto": kproto,
    "scaler": scaler,
    "cat_cols": cat_cols,
    "num_cols": num_cols,
    "cat_idx": cat_idx,
    "cluster_labels": cluster_labels,
    "num_medians": num_medians,
    "cluster_labels_name_map": cluster_name_map,
}

model_path = save_path / "user_cluster_kproto.joblib"
dump(artifacts, model_path)
df["TRAVELER_ID"] = traveler_id
df['cluster'] = df_scaled["cluster"]
df['cluster_name'] = df['cluster'].map(cluster_name_map)
# df = df.drop(columns=drop_cols)
df.to_csv(save_path / "rating_user_info_all_cluster.csv")

print("✅ 모델 아티팩트 저장 완료:", model_path)
print("✅ 클러스터 데이터 저장 완료:", data_path / "rating_user_info_all_cluster.csv")


✅ 모델 아티팩트 저장 완료: C:\Users\User\Desktop\xodls\through_pjt_ai\AI 모델\1.모델소스코드\3.여행루트추천\model\user_cluster_kproto.joblib
✅ 클러스터 데이터 저장 완료: C:\Users\User\Desktop\xodls\through_pjt_ai\AI 모델\1.모델소스코드\1.여행로그 장소 추천 고도화\data\rating_user_info_all_cluster.csv


In [65]:
from joblib import load

model_path = save_path / "user_cluster_kproto.joblib"
artifacts = load(model_path)

kproto = artifacts["kproto"]
scaler = artifacts["scaler"]
cat_cols = artifacts["cat_cols"]
num_cols = artifacts["num_cols"]
cat_idx = artifacts["cat_idx"]
cluster_labels = artifacts["cluster_labels"]
num_medians = artifacts["num_medians"]

print("✅ 모델 아티팩트 로드 완료")


✅ 모델 아티팩트 로드 완료


In [66]:
import pandas as pd
import numpy as np

def assign_new_user(user_dict):
    """
    완전 신규 유저에 대해 군집 ID와 라벨을 반환하는 함수.

    user_dict 예시:
    {
        "GENDER": "남",
        "AGE_GRP": "30",
        "MARR_STTS": "1",
        "JOB_NM": "3",
        "INCOME": 4,
        "TRAVEL_STYL_1": "2",
        "TRAVEL_STATUS_RESIDENCE": "서울특별시",
        "TRAVEL_STATUS_ACCOMPANY": "2인 여행(가족 외)",
        "TRAVEL_MOTIVE_1": "7",
        "TRAVEL_NUM": 3,
        "TRAVEL_COMPANIONS_NUM": 1,
        "MONTH": "8",
        "SEASON": "summer",
        "HOW_LONG": 3,
    }
    """
    # 1) dict → DataFrame
    new_df = pd.DataFrame([user_dict])

    # 2) 없는 컬럼 자동 추가 (완전 신규 유저 대비)
    for col in cat_cols:
        if col not in new_df.columns:
            new_df[col] = "Unknown"
    for col in num_cols:
        if col not in new_df.columns:
            new_df[col] = np.nan

    # 3) 결측값 처리
    new_df[cat_cols] = new_df[cat_cols].fillna("Unknown")
    for c in num_cols:
        new_df[c] = new_df[c].fillna(num_medians[c])

    # 4) 타입 정리
    new_df[cat_cols] = new_df[cat_cols].astype(str)

    # 5) 스케일링
    new_df[num_cols] = scaler.transform(new_df[num_cols])

    # 6) numpy 변환 (dtype=object 필수)
    new_np = new_df[cat_cols + num_cols].to_numpy(dtype=object)

    # 7) 군집 예측
    cluster = int(kproto.predict(new_np, categorical=cat_idx)[0])
    label = cluster_labels[cluster]

    return cluster, label


In [67]:
example_user = {
    "GENDER": "남",
    "AGE_GRP": "30",
    "MARR_STTS": "1",
    "JOB_NM": "3",
    "INCOME": 4,
    "TRAVEL_STYL_1": "2",
    "TRAVEL_STATUS_RESIDENCE": "서울특별자치도",
    "TRAVEL_STATUS_ACCOMPANY": "2인 여행(가족 외)",
    "TRAVEL_MOTIVE_1": "7",
    "TRAVEL_NUM": 3,
    "TRAVEL_COMPANIONS_NUM": 1,
    "MONTH": "8",
    "SEASON": "summer",
    "HOW_LONG": 3,
}

cluster_id, label = assign_new_user(example_user)

print("신규 유저 군집:", cluster_id)
print("군집 라벨:", label)
print("군집 라벨 이름:", cluster_name_map[cluster_id])


신규 유저 군집: 4
군집 라벨: 30대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.6일 여행자
군집 라벨 이름: 느슨하게 떠나는 듀오 무드러
